### This is the Notebook for In-Class Coding 2 - Logic Design - Finite State Machines

In this lecture, we will learn
<ol>
    <li>Develop and test a 1-bit Full Adder</li>
    <li>Use Recursion to develop a 4-bit Full Adder with connected gates</li>
    <li>Use a register to store the state value for a Finite State Machine</li>
</ol>
<br />
First, we will run the code blocks we developed in Mondays's lecture:

In [ ]:
# You will do this library import with every PyRTL assignment
from pyrtl import *
import pyrtl

In [ ]:
def two_input_and( a_input, b_input ):
    
    # Create the wire out and put a & b on that 
    y_output = a_input & b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [ ]:
def two_input_or( a_input, b_input ):
    
    # Create the wire out and put a | b on that 
    y_output = a_input | b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [ ]:
def two_input_xor( a_input, b_input ):
    
    # Create the wire out and put a ^ b on that 
    y_output = a_input ^ b_input

    # Use assert to ensure that the signals are one bit
    return y_output

## Classic Advanced Circuit: Adder

Now we will build the full adder that we developed together in class on Mnday

Recall that the logic <b>equations</b> we developed were as follows:
<ol>
    <li><code>Sum = a ^ b ^ Cin</code></li>
    <li><code>Cout = (a & b) | (Cin & (a ^ b))</code></li>
</ol>

The <b>logic diagram</b> and <b>truth table</b> are presented below:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Full-adder_logic_diagram.svg/400px-Full-adder_logic_diagram.svg.png"></center> <br />

| A | B | Cin | Sum | Cout |
|---|---|---|---|---|
|0|0|0|0|0|
|0|0|1|1|0|
|0|1|0|1|0|
|0|1|1|0|1|
|1|0|0|1|0|
|1|0|1|0|1|
|1|1|0|0|1|
|1|1|1|1|1|

In [ ]:
def sum( a, b, c_in ):
    
    # In-Class: Return the sum using previously design cells
    # Could also simply do a ^ b ^ cin
    return two_input_xor( two_input_xor(a, b), c_in )

In [ ]:
def carry_out( a, b, c_in ):
    
    # In-Class: Return the carry out
    return (a & b) | ( c_in & (a ^ b) )

In [ ]:
def full_adder( a, b, c_in ):
    
    # In-Class: Calculate the sum
    output_sum = sum( a, b, c_in )

    # In-Class: Calculate the carry out
    output_c_out = carry_out( a, b, c_in )
    
    # Return both
    return output_sum, output_c_out

In [ ]:
def full_adder_simulate():
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # In-Class: Step 2 - Create the input and ouput wires
    a, b, c_in = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b'), pyrtl.Input(1, 'c_in')
    output_sum = pyrtl.Output(1, 'output_sum')
    output_c_out = pyrtl.Output(1, 'output_c_out')
    
    # In-Class: Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = full_adder( a, b, c_in )
    
    # In-Class: Step 3-b Assign to a wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs =    [0,0,0,0,1,1,1,1]
    b_inputs =    [0,0,1,1,0,0,1,1]
    c_in_inputs = [0,1,0,1,0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, len(a_inputs)):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [ ]:
# Call the function
full_adder_simulate()

### Combining Concepts: Using Recursion to Create a Full Adder

When we create a circuit in PyRTL, we cannot use loop to create the signals, only the input signals. But if there is repetition, we can use <b>recursion</b>!

In [ ]:
def ripple_carry_adder( a, b, c_in ):
    
    # Base Case: When a is a length of 1, we put the first full adder
    if len(a) == 1:
        sum_result, c_out = full_adder(a, b, c_in )
    
    # Recursive Case
    else:

        # Get the "least significant" bit and the rippecarry to the next full adder
        ls_bit, ripplecarry = full_adder(a[0], b[0], c_in)
        
        # Recursively call the remaining bits to get the "most significant" bits and the current carry out
        ms_bits, c_out = ripple_carry_adder( a[1:], b[1:], ripplecarry )
        
        # Use pyrtl.concat Concatenate the results on the recursive call back up
        sum_result = pyrtl.concat(ms_bits, ls_bit)

    # Return the results                                      
    return sum_result, c_out

In [ ]:
def ripple_carry_sim( ):
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # In-Class: Step 2 - Create the input and ouput wires for a 4-bit adder
    a, b, c_in = pyrtl.Input(4, 'a'), pyrtl.Input(4, 'b'), pyrtl.Input(1, 'c_in')
    output_sum = pyrtl.Output(4, 'output_sum')
    output_c_out = pyrtl.Output(1, 'output_c_out')    
    
    
    # In-Class:Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = ripple_carry_adder(a, b, c_in)
    
    # In-Class: Step 3-b Assign to a bus wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [ 0, 3, 10, 2, 15, 3, 14 ]
    b_inputs = [ 4, 6, 6, 11, 15, 12, 1 ]
    c_in_inputs = [0, 0, 0, 0, 0, 0, 0]

    # Step 6 - Loop through and simuluate
    for value in range( 0, len(a_inputs) ):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [ ]:
ripple_carry_sim( )

# Finite State Machine Example

Recall from the reading assignment: We can write digital logic that acts like a graph called a <b>Finite State Machine</b>. A <b>Finite State Machine</b> consists of:<br />
<ol>
    <li>A set of states (represented as nodes)</li>
    <li>An initial state</li>
    <li>A set of transitions between states (represented by edges)</li>
    <li>A set of control input signals</li>
</ol>

### Now, we can define states. 

Using the same nodes and transitions as above, Here are the outputs we will create for our new FSM:

<ol>
    <li><b>State Name:</b><code>q0</code>, <b>Outputs:</b> <code>out_0 = 1</code>, <code>out_1 = a & b</code></li>
    <li><b>State Name:</b><code>q1</code>, <b>Outputs:</b> <code>out_0 = a | b | c</code>, <code>out_1 = a ^ b</code></li>
    <li><b>State Name:</b><code>q2</code>, <b>Outputs:</b> <code>out_0 = a & b & c</code>, <code>out_1 = 0</code></li>
    <li><b>State Name:</b><code>q3</code>, <b>Outputs:</b> <code>out_0 = a ^ b</code>, <code>out_1 = b & c</code></li>
    <li><b>State Name:</b><code>q4</code>, <b>Outputs:</b> <code>out_0 = 0</code>, <code>out_1 = 1</code></li>
</ol>

<img src = "https://mmorri22-nd.neocities.org/images/lec26_fsm.jpg" width=700 height=700>

### Test Suite

Here is the truth table for our Finite State Machine

| State | out_0 | out_1 | next_state 0 | next_state 1 |
|---|---|---|---|---|
|q0| 1 | a & b | q1 | q2 |
|q1| a or b or c | a ^ b | q1 | q3 |
|q2| a & b & c | 0 | q4 | q2 |
|q3| a ^ b | b & c| q4 | q2 |
|q4| 0 | 1 |q1 | q3 |

Based on these functions and the FSM Diagram, here are the <i>expected</i> outputs:


| Start State | a | b | c | out_0 | out_1 | Control | Next State |
|---|---|---|---|---|---|---|---|
|<b><font color="purple">q0</font></b>|1|1|1|<b><font color="red">1</font></b>| a & b = <b><font color="blue">1</font></b>| 1 | q2 |
|<b><font color="purple">q2</font></b>|1|0|0|a & b & c = <b><font color="red">0</font></b>| <b><font color="blue">0</font></b> | 1 | q2 |
|<b><font color="purple">q2</font></b>|1|1|1|a & b & c = <b><font color="red">1</font></b>| <b><font color="blue">0</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|0|1|0|<b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|0|1|1|a ^ b = <b><font color="red">1</font></b>| b & c = <b><font color="blue">1</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|1|0|0|<b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 0 | q1 |
|<b><font color="purple">q1</font></b>|1|0|0| a or b or c = <b><font color="red">1</font></b> | a ^ b = <b><font color="blue">1</font></b> | 0 | q1 |
|<b><font color="purple">q1</font></b>|1|1|0| a or b or c = <b><font color="red">1</font></b> | a ^ b = <b><font color="blue">0</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|1|1|1| a ^ b = <b><font color="red">0</font></b>| b & c = <b><font color="blue">1</font></b> | 1 | q2 |

### Additional Functions

Click through these functions that we will use inside each state:

In [ ]:
def three_input_and( a_input, b_input, c_input ):
    
    # Create the wire out and put a & b & c on that 
    y_output = a_input & b_input & c_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [ ]:
def three_input_or( a_input, b_input, c_input ):
    
    # Create the wire out and put a | b | c on that 
    y_output = a_input | b_input | c_input

    # Use assert to ensure that the signals are one bit
    return y_output

### First, define each state in the FSM

To promote modularity, and reduce the challenges of troubleshooting, keep the "current state operations" and the "next state calculations separate.

In [ ]:
# For this function, we assume the initial state has already been executed impemented

def curr_state_op( curr_state, a_input, b_input, c_input ):
    
    # In-Class: Must include with pyrtl.conditional_assignment
    with pyrtl.conditional_assignment:
        
        # In-Class: Create two intermediate WireVectors of bitwidth 1
        state_out_0 = pyrtl.WireVector(1)
        state_out_1 = pyrtl.WireVector(1)
        
        # In-Class: Perform the transition
        
        # State q0
        with curr_state == 0:
            state_out_0 |= 1
            state_out_1 |= two_input_and( a_input, b_input )
        
        # State q1
        with curr_state == 1:
            state_out_0 |= three_input_or( a_input, b_input, c_input )
            state_out_1 |= two_input_xor( a_input, b_input )
        
        # State q2
        with curr_state == 2:
            state_out_0 |= three_input_and( a_input, b_input, c_input )
            state_out_1 |= 0

        # State q3
        with curr_state == 3:
            state_out_0 |= two_input_xor( a_input, b_input )
            state_out_1 |= two_input_and( b_input, c_input )

        # State q4
        with curr_state == 4:
            state_out_0 |= 0
            state_out_1 |= 1
            
        # Return both wire outputs
        return state_out_0, state_out_1

In [ ]:
# For this function, we assume the initial state has already been executed impemented

def next_state( curr_state, control_signal ):
    
    # Every time you perform a conditional assignment
    with pyrtl.conditional_assignment:
            
        # Perform the transition
        
        # State q0
        with curr_state == 0:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 2
                
        # State q1
        with curr_state == 1:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 3
                
        # State q2
        with curr_state == 2:
            
            with control_signal == 0:
                curr_state.next |= 4
                
            with control_signal == 1:
                curr_state.next |= 2
                
        # State q3
        with curr_state == 3:
            
            with control_signal == 0:
                curr_state.next |= 4
                
            with control_signal == 1:
                curr_state.next |= 2
                

        # State q4
        with curr_state == 4:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 3
        
        # Return the updated memory
        return curr_state.next

In [ ]:
def example_fsm_simulate( ):
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # Step 2 - Create the input and ouput wires
    
    # Inputs 
    control_signal = pyrtl.Input(2, 'control_signal')
    a_in = pyrtl.Input(1, 'a_in')
    b_in = pyrtl.Input(1, 'b_in')
    c_in = pyrtl.Input(1, 'c_in')
    
    # Outputs
    output_0 = pyrtl.Output(1, 'output_0')
    output_1 = pyrtl.Output(1, 'output_1')
    
    
    # In-Class: Define Register. Needs to have the number of bits requried to hold the value 
    # Since we have five states, we need three bits
    curr_state = pyrtl.Register(3, 'curr_state')

    
    # In-Class Step 3 - Save to an intermediate value using example_fsm the function
    inter_0, inter_1 = curr_state_op( curr_state, a_in, b_in, c_in )
    
    output_0 <<= inter_0
    output_1 <<= inter_1
    
    # 3b - Map intermediate states to output signals
    curr_state = next_state( curr_state, control_signal )
    
    # Step 4 - Start the design simulation
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    control_signals = [1,1,0,1,0,0,0,1,1]
    a_inputs        = [1,1,1,0,0,1,1,1,1]
    b_inputs        = [1,0,1,1,1,0,0,1,1]
    c_inputs        = [1,0,1,0,1,0,0,0,1]
    
    
    # Step 6 - Loop through and simuluate
    
    for value in range(0, len(a_inputs)):

        sim.step({
            
            'control_signal' : control_signals[value],
            'a_in' : a_inputs[value],
            'b_in' : b_inputs[value],
            'c_in' : c_inputs[value] 
            
        })
    
    # Step 7- Render the trace
    sim.tracer.render_trace()

In [ ]:
example_fsm_simulate()

### Check: Compare the generated waveform with the table we developed earlier

| Start State | a | b | c | out_0 | out_1 | Control | Next State |
|---|---|---|---|---|---|---|---|
|<b><font color="purple">q0</font></b>|1|1|1|<b><font color="red">1</font></b>| a & b = <b><font color="blue">1</font></b>| 1 | q2 |
|<b><font color="purple">q2</font></b>|1|0|0|a & b & c = <b><font color="red">0</font></b>| <b><font color="blue">0</font></b> | 1 | q2 |
|<b><font color="purple">q2</font></b>|1|1|1|a & b & c = <b><font color="red">1</font></b>| <b><font color="blue">0</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|0|1|0|<b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|0|1|1|a ^ b = <b><font color="red">1</font></b>| b & c = <b><font color="blue">1</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|1|0|0|<b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 0 | q1 |
|<b><font color="purple">q1</font></b>|1|0|0| a or b or c = <b><font color="red">1</font></b> | a ^ b = <b><font color="blue">1</font></b> | 0 | q1 |
|<b><font color="purple">q1</font></b>|1|1|0| a or b or c = <b><font color="red">1</font></b> | a ^ b = <b><font color="blue">0</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|1|1|1| a ^ b = <b><font color="red">0</font></b>| b & c = <b><font color="blue">1</font></b> | 1 | q2 |